In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import os
import tensorflow as tf

In [3]:
work_dir = './dogs_cats/data/'
image_height, image_width = 150, 150
train_dir = os.path.join(work_dir, 'train')
test_dir = os.path.join(work_dir, 'test')

no_classes = 2
no_validation = 800
epochs = 2
batch_size = 200
no_train = 2000
no_test = 800
input_shape = (image_height, image_width, 3)
epoch_steps = no_train // batch_size
test_steps = no_test // batch_size

In [4]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)

In [5]:
model = tf.keras.applications.VGG16(include_top=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
58892288/58889256 [==============================] - 22s 0us/step


In [6]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [7]:
train_images = generator.flow_from_directory(train_dir,
                                             batch_size=batch_size,
                                             target_size=(image_width, image_height),
                                             class_mode=None,
                                             shuffle=False)

Found 2000 images belonging to 2 classes.


In [8]:
train_bottleneck_features = model.predict_generator(train_images, epoch_steps)

In [9]:
test_images = generator.flow_from_directory(test_dir,
                                             batch_size=batch_size,
                                             target_size=(image_width, image_height),
                                             class_mode=None,
                                             shuffle=False)

Found 800 images belonging to 2 classes.


In [10]:
test_bottleneck_features = model.predict_generator(test_images, test_steps)

In [20]:
import numpy as np
train_labels = np.array([0] * int(no_train / 2) + [1] * int(no_train / 2))
test_labels = np.array([0] * int(no_test / 2) + [1] * int(no_test / 2))

In [18]:
from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [12]:
train_bottleneck_features.shape[1:]

(4, 4, 512)

In [22]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=train_bottleneck_features.shape[1:]))
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1, activation='softmax'))
model.compile(loss=tf.keras.losses.binary_crossentropy,
             optimizer=tf.keras.optimizers.Adam(),
             metrics=['accuracy'])

In [23]:
model.fit(train_bottleneck_features,
         train_labels,
         batch_size=batch_size,
         epochs=epochs,
         validation_data=(test_bottleneck_features, test_labels))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2000 samples, validate on 800 samples
Epoch 1/2
2000/2000 [==============================] - 1s 281us/sample - loss: 7.6666 - acc: 0.5000 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 2/2
2000/2000 [==============================] - 0s 230us/sample - loss: 7.6666 - acc: 0.5000 - val_loss: 7.6246 - val_acc: 0.5000
